In [1]:
# Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import json
import os

# Load the model
model_path = "model.h5"  # Path to the model file
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully.")
else:
    print(f"Model file '{model_path}' not found. Please ensure the file exists.")

# Define a function for predictions
def predict_image(image_path, model, class_indices, threshold=0.8):
    """
    Predict the class of an image using a pre-trained model.

    Args:
        image_path (str): Path to the input image.
        model (tf.keras.Model): Pre-trained model.
        class_indices (dict): Mapping of class indices to labels.

    Returns:
        dict: Prediction results in JSON format.
    """
    try:
        # Load and preprocess the image
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Normalize the image

        # Perform prediction
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        confidence = predictions[0][predicted_class]

        if confidence < threshold:
            predicted_class = 6  # Assign to "Other" if confidence is below threshold

        # Convert to JSON format
        label_mapping = {0: "HDPE", 1: "LDPE", 2: "PET", 3: "PP", 4: "PS", 5: "PVC", 6: "Other"}
        result = {
            "predicted_class": label_mapping.get(predicted_class, "Other"),
            "confidence": float(confidence),
            "all_predictions": {
                label_mapping[i]: float(predictions[0][i]) if i in label_mapping else "N/A"
                for i in range(len(predictions[0]))
            },
        }
        return result
    except Exception as e:
        return {"error": str(e)}

# Define a function to save JSON output
def save_json_output(output, json_file_path):
    """
    Save prediction output to a JSON file.

    Args:
        output (dict): Prediction output in dictionary format.
        json_file_path (str): Path to save the JSON file.
    """
    try:
        with open(json_file_path, 'w') as json_file:
            json.dump(output, json_file, indent=4)
        print(f"Output saved to '{json_file_path}'.")
    except Exception as e:
        print(f"Error saving JSON file: {str(e)}")

# Class indices
# Mapping class indices from model training
# Ensure your model outputs match this structure
class_indices = {"HDPE": 0, "LDPE": 1, "PET": 2, "PP": 3, "PS": 4, "PVC": 5, "Other": 6}

# Predict an image and save to JSON
# Example usage
image_path = "kantong plastik putih_3.jpg"  # Replace with the path to your test image
json_output_path = "prediction_output.json"  # Replace with desired JSON file path

if os.path.exists(image_path):
    prediction = predict_image(image_path, model, class_indices)
    print("Prediction output in JSON format:")
    print(json.dumps(prediction, indent=4))  # Print prediction to console
    save_json_output(prediction, json_output_path)  # Save to JSON file
else:
    print(f"Image file '{image_path}' not found.")


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction output in JSON format:
{
    "predicted_class": "LDPE",
    "confidence": 0.9989727735519409,
    "all_predictions": {
        "HDPE": 0.00020431716984603554,
        "LDPE": 0.9989727735519409,
        "PET": 1.973083635675721e-05,
        "PP": 3.982330599683337e-05,
        "PS": 0.00023635975958313793,
        "PVC": 0.0005270957481116056
    }
}
Output saved to 'prediction_output.json'.
